> **Parte a)**

In [2]:
import pandas as pd

link_train = 'https://raw.githubusercontent.com/csaldias/tarea3-RedesNeuronales/master/train_Q-A.csv'
link_test  = 'https://raw.githubusercontent.com/csaldias/tarea3-RedesNeuronales/master/test_Q.csv'

df_train = pd.read_csv(link_train)
df_train.dropna(inplace=True)
df_test = pd.read_csv(link_test)
df_train.head()

,id,question,answer
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s


In [3]:
df_test.head()

,id,question
0,56ddde6b9a695914005b9628,In what country is Normandy located?
1,56ddde6b9a695914005b9629,When were the Normans in Normandy?
2,56ddde6b9a695914005b962a,From which countries did the Norse originate?
3,56ddde6b9a695914005b962b,Who was the Norse leader?
4,56ddde6b9a695914005b962c,What century did the Normans first gain their ...


In [4]:
print("Hay {} ejemplos disponibles para entrenar.".format(df_train.shape[0]))
print("Hay {} ejemplos disponibles para predecir.".format(df_test.shape[0]))

Hay 86821 ejemplos disponibles para entrenar.
Hay 11873 ejemplos disponibles para predecir.


> **Parte b)**

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize

train_questions = [word_tokenize(sentence.lower()) for sentence in df_train["question"]] #or processing
test_questions  = [word_tokenize(sentence.lower()) for sentence in df_test["question"]]
train_answers   = [word_tokenize(sentence) for sentence in df_train["answer"]]

In [7]:
print(train_answers[30])

['2000s']


> **Parte c)**

In [8]:
vocab_answer = set()
for sentence in train_answers:
    for word in sentence:
        vocab_answer.add(word)
vocab_answer = ["#end"]+ list(vocab_answer)
print('posibles palabras para respuestas :', len(vocab_answer))
vocabA_indices = {c: i for i, c in enumerate(vocab_answer)}
indices_vocabA = {i: c for i, c in enumerate(vocab_answer)}

posibles palabras para respuestas : 47423


In [9]:
vocab_question = set()
for sentence in train_questions:
    for word in sentence:
        vocab_question.add(word)
for sentence in test_questions:
    for word in sentence:
        vocab_question.add(word)
vocab_question = ["#end"]+ list(vocab_question)
print('posibles palabras para preguntas :', len(vocab_question))
vocabQ_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQ = {i: c for i, c in enumerate(vocab_question)}

posibles palabras para preguntas : 42060


> **Parte d)**

In [0]:
#input and output to onehotvector
X_answers = [[vocabA_indices[palabra] for palabra in sentence] for sentence in train_answers]
Xtrain_question = [[vocabQ_indices[palabra] for palabra in sentence] for sentence in train_questions]#same for train question
Xtest_question = [[vocabQ_indices[palabra] for palabra in sentence] for sentence in test_questions]#same for test question

In [11]:
import numpy as np
max_input_lenght  = np.max(list(map(len, train_questions)))
max_output_lenght = np.max(list(map(len, train_answers)))+1

from keras.preprocessing import sequence
Xtrain_question = sequence.pad_sequences(Xtrain_question, maxlen=max_input_lenght, padding='post', value=0)
Xtest_question  = sequence.pad_sequences(Xtest_question, maxlen=max_input_lenght, padding='post', value=0)
X_answers       = sequence.pad_sequences(X_answers, maxlen=max_output_lenght, padding='post', value=vocabA_indices["#end"])

Using TensorFlow backend.


In [12]:
print("Dimensionalidad Xtrain_question:",Xtrain_question.shape)
print("Dimensionalidad X_answers:\t",X_answers.shape)
print("Dimensionalidad Xtest_question:\t",Xtest_question.shape)

Dimensionalidad Xtrain_question: (86821, 60)
Dimensionalidad X_answers:	 (86821, 47)
Dimensionalidad Xtest_question:	 (11873, 60)


> **Parte e)**

In [0]:
#Encoder-Decoder modelo
from keras.layers import Input,RepeatVector,TimeDistributed,Dense,Embedding,Flatten,Activation,Permute,Lambda,CuDNNLSTM
from keras.models import Model
from keras import backend as K

lenght_output = max_output_lenght
hidden_dim = 128

In [0]:
embedding_vector = 64 
encoder_input = Input(shape=(max_input_lenght,))
embedded = Embedding(input_dim=len(vocabQ_indices),output_dim=embedding_vector,input_length=max_input_lenght)(encoder_input)
encoder = CuDNNLSTM(hidden_dim, return_sequences=True)(embedded)

In [0]:
# compute T' importance for each step T
attention = TimeDistributed(Dense(max_output_lenght, activation='tanh'))(encoder)
#softmax a las antenciones sobre todo T
attention = Permute([2, 1])(attention)
attention = Activation('softmax')(attention) 
attention = Permute([2, 1])(attention)

In [0]:
# apply the attention to encoder
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)
sent_representation = Lambda(attention_multiply)([encoder, attention])
decoder = CuDNNLSTM(hidden_dim, return_sequences=True)(sent_representation)
probabilities = TimeDistributed(Dense(len(vocab_answer), activation="softmax"))(decoder)

In [17]:
model = Model(encoder_input,probabilities)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 64)       2691840     input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        (None, 60, 128)      99328       embedding_1[0][0]                
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 60, 47)       6063        cu_dnnlstm_1[0][0]               
__________________________________________________________________________________________________
permute_1 

> **Parte f)**

In [19]:
X_answers = X_answers.reshape(X_answers.shape[0],X_answers.shape[1],1)
X_answers.shape
model.fit(Xtrain_question,X_answers,epochs=10,batch_size=128,validation_split=0.2)

Train on 69456 samples, validate on 17365 samples
Epoch 1/10
69456/69456 [==============================] - 349s 5ms/step - loss: 0.7267 - val_loss: 0.7981
Epoch 2/10
15104/69456 [=====>........................] - ETA: 4:07 - loss: 0.6984

69456/69456 [==============================] - 349s 5ms/step - loss: 0.6915 - val_loss: 0.8024
Epoch 3/10
50432/69456 [====================>.........] - ETA: 1:26 - loss: 0.6703

69456/69456 [==============================] - 349s 5ms/step - loss: 0.6713 - val_loss: 0.8078
Epoch 4/10
64640/69456 [==========================>...] - ETA: 21s - loss: 0.6542

69456/69456 [==============================] - 349s 5ms/step - loss: 0.6547 - val_loss: 0.8129
Epoch 5/10
69456/69456 [==============================] - 349s 5ms/step - loss: 0.6409 - val_loss: 0.8244
Epoch 6/10
  384/69456 [..............................] - ETA: 5:15 - loss: 0.6071

69456/69456 [==============================] - 348s 5ms/step - loss: 0.6291 - val_loss: 0.8322
Epoch 7/10
44416/69456 [==================>...........] - ETA: 1:53 - loss: 0.6204

69456/69456 [==============================] - 349s 5ms/step - loss: 0.6181 - val_loss: 0.8415
Epoch 8/10
62208/69456 [=========================>....] - ETA: 32s - loss: 0.6076

69456/69456 [==============================] - 349s 5ms/step - loss: 0.6081 - val_loss: 0.8471
Epoch 9/10
69376/69456 [============================>.] - ETA: 0s - loss: 0.5989

69456/69456 [==============================] - 349s 5ms/step - loss: 0.5990 - val_loss: 0.8538
Epoch 10/10
51328/69456 [=====================>........] - ETA: 1:22 - loss: 0.5896

69456/69456 [==============================] - 348s 5ms/step - loss: 0.5903 - val_loss: 0.8655
